In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
%matplotlib inline
import datetime
#plt.style.use('ggplot')
#plt.rcParams['figure.figsize'] = (12, 6)

# Set the global default size of matplotlib figures
plt.rc('figure', figsize=(10, 5))

In [2]:
import os
import re
path = 'API_Data/'

def getfiles(path):
    a = [s for s in os.listdir(path)
         if os.path.isfile(os.path.join(path, s))]
    a.sort(key=lambda s: os.path.getmtime(os.path.join(path, s)))
    return a

In [3]:
files = getfiles(path)
files = [f for f in files if re.match(r'2016',f)]

In [4]:
import ujson
from pandas.io.json import json_normalize

howManyFiles = len(files)

dataList = [[] for i in range(howManyFiles)]
dataList[0]

print str(howManyFiles) +' files in the directory'
for i in range(howManyFiles):
    fname = ('API_Data/' +files[i]).strip()
    with open(fname,'r') as f:
        dataList[i] = f.read()
        if i % 100 == 0: print i,

18850 files in the directory
0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 4000 4100 4200 4300 4400 4500 4600 4700 4800 4900 5000 5100 5200 5300 5400 5500 5600 5700 5800 5900 6000 6100 6200 6300 6400 6500 6600 6700 6800 6900 7000 7100 7200 7300 7400 7500 7600 7700 7800 7900 8000 8100 8200 8300 8400 8500 8600 8700 8800 8900 9000 9100 9200 9300 9400 9500 9600 9700 9800 9900 10000 10100 10200 10300 10400 10500 10600 10700 10800 10900 11000 11100 11200 11300 11400 11500 11600 11700 11800 11900 12000 12100 12200 12300 12400 12500 12600 12700 12800 12900 13000 13100 13200 13300 13400 13500 13600 13700 13800 13900 14000 14100 14200 14300 14400 14500 14600 14700 14800 14900 15000 15100 15200 15300 15400 15500 15600 15700 15800 15900 16000 16100 16200 16300 16400 16500 16600 16700 16800 16900 17000 17100 17200 17300 17400 17500 17600 17700 17800 17900 180

In [5]:
import pandas as pd
all_dfs = pd.DataFrame()
j = 0
for i in np.random.randint(0,howManyFiles,300): #range(howManyFiles):
    j += 1
    if j % 100 == 0: print j,
    #if i % 50 == 0: print i,
    try:    
        r = ujson.loads(dataList[i])
        time = r['executionTime']
        data = r['stationBeanList']
        len(data) # 508 docks

        df = json_normalize(r['stationBeanList']) # convert to DF
        df['time'] = r['executionTime']
    
        all_dfs = all_dfs.append(df)
    except:
        print "some trouble"
        pass

100 200 300


In [6]:
all_dfs.shape

(152494, 19)

In [7]:
all_dfs[all_dfs['availableDocks'] == 0].columns

Index([             u'altitude',        u'availableBikes',
              u'availableDocks',                  u'city',
                          u'id',              u'landMark',
       u'lastCommunicationTime',              u'latitude',
                    u'location',             u'longitude',
                  u'postalCode',            u'stAddress1',
                  u'stAddress2',           u'stationName',
                   u'statusKey',           u'statusValue',
                 u'testStation',            u'totalDocks',
                        u'time'],
      dtype='object')

In [8]:
def splitDate(col, df):
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    month = []
    day = []
    year = []
    hour = []
    minute = []
    day_of_week = []
    
    d = []
    for r in df[col]:
        tmobj = datetime.datetime.strptime(r, '%Y-%m-%d %I:%M:%S %p')

        month.append(tmobj.month)
        year.append(tmobj.year)
        hour.append(tmobj.hour)
        minute.append(tmobj.minute)
        
        if (tmobj.day not in d):
            d.append(tmobj.day)
            print len(d),
        day.append(tmobj.day)
        day_of_week.append(days[tmobj.weekday()]) 
    return month, year, hour, minute, day, day_of_week

def add_columns(df):
    month, year, hour, minute, day, day_of_week = splitDate('lastCommunicationTime', df)
    df['year'] = year
    df['month'] = month
    df['hour'] = hour
    df['minute'] = minute
    df['day'] = day
    df['day_of_week'] = day_of_week
    return df

In [9]:
a = '2016-01-14 06:51:19 PM'

In [10]:
t = datetime.datetime.strptime(a, '%Y-%m-%d %I:%M:%S %p')

In [11]:
t.hour

18

In [12]:
all_dfs.iloc[6000]['lastCommunicationTime']

u'2016-01-18 07:38:55 AM'

In [13]:
all_dfs = add_columns(all_dfs)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19


In [14]:
all_dfs.columns

Index([             u'altitude',        u'availableBikes',
              u'availableDocks',                  u'city',
                          u'id',              u'landMark',
       u'lastCommunicationTime',              u'latitude',
                    u'location',             u'longitude',
                  u'postalCode',            u'stAddress1',
                  u'stAddress2',           u'stationName',
                   u'statusKey',           u'statusValue',
                 u'testStation',            u'totalDocks',
                        u'time',                  u'year',
                       u'month',                  u'hour',
                      u'minute',                   u'day',
                 u'day_of_week'],
      dtype='object')

In [15]:
all_dfs = all_dfs[all_dfs['year'] == 2016]

* **~2% of the time there are no docks available**

In [16]:
all_dfs[all_dfs['availableDocks'] == 0].shape[0] / float(all_dfs.shape[0])*100

1.9558868228733035

In [17]:
no_docks = all_dfs[all_dfs['availableDocks'] == 0]

In [18]:
no_docks

,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,...,statusValue,testStation,totalDocks,time,year,month,hour,minute,day,day_of_week
124,,0,0,,343,,2016-01-24 10:44:36 AM,40.697940,,-73.969868,...,Not In Service,False,23,2016-01-24 10:44:53 AM,2016,1,10,44,24,Sunday
151,,0,0,,373,,2016-01-24 10:41:34 AM,40.693317,,-73.953820,...,Not In Service,False,19,2016-01-24 10:44:53 AM,2016,1,10,41,24,Sunday
220,,0,0,,453,,2016-01-24 10:42:52 AM,40.744751,,-73.999154,...,Not In Service,False,39,2016-01-24 10:44:53 AM,2016,1,10,42,24,Sunday
243,,0,0,,477,,2016-01-24 10:42:33 AM,40.756405,,-73.990026,...,Not In Service,False,59,2016-01-24 10:44:53 AM,2016,1,10,42,24,Sunday
279,,0,0,,514,,2016-01-24 10:42:57 AM,40.760875,,-74.002777,...,Not In Service,False,53,2016-01-24 10:44:53 AM,2016,1,10,42,24,Sunday
294,,0,0,,529,,2016-01-24 10:43:08 AM,40.757570,,-73.990985,...,Not In Service,False,41,2016-01-24 10:44:53 AM,2016,1,10,43,24,Sunday
485,,0,0,,3214,,2016-01-22 07:09:41 PM,40.712774,,-74.036486,...,Not In Service,False,22,2016-01-24 10:44:53 AM,2016,1,19,9,22,Friday
8,,30,0,,128,,2016-01-17 07:58:16 AM,40.727103,,-74.002971,...,In Service,False,30,2016-01-17 07:59:51 AM,2016,1,7,58,17,Sunday
90,,36,0,,306,,2016-01-17 07:58:40 AM,40.708235,,-74.005301,...,In Service,False,37,2016-01-17 07:59:51 AM,2016,1,7,58,17,Sunday
122,,17,0,,341,,2016-01-17 07:57:40 AM,40.717821,,-73.976289,...,In Service,False,19,2016-01-17 07:59:51 AM,2016,1,7,57,17,Sunday


In [ ]:
all_dfs.groupby('id')['availableDocks'].apply(lambda x: x == 0)

In [ ]:
all_dfs.groupby(['id','hour'])['availableDocks']

In [ ]:
temp = no_docks.groupby(['id','hour']).sum()

In [ ]:
temp.head()

In [ ]:
h

In [ ]:
byhour = [temp.loc[temp.index.get_level_values('hour') == h].shape[0] for h in range(24)]

In [ ]:
plt.plot(byhour,'o-')